# Chapter 6: Teaching Machines to See

## Image Classification

<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/thushv89/manning_tf2_in_action/blob/master/Ch06/6.1.Transformer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>

## Importing necessary libraries

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import requests
print(tf.__version__)
import zipfile
import requests
import os
import time
import pandas as pd
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, AvgPool2D, Dense, Concatenate, Flatten, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
import numpy as np
from PIL import Image
import tensorflow.keras.backend as K
import pickle
from tensorflow.keras.models import load_model, Model

random_seed = 4321
np.random.seed(random_seed)
seed=4321
random.seed(seed)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass

2.1.0


## Downloading data

For this chapter, we're going to use the tiny-imagenet dataset (200 categories of objects). This is a simplified version of the bigger and harder imagenet dataset (1000 categories of objects).

In [2]:
import os
import requests
import zipfile

# Retrieve the data
if not os.path.exists(os.path.join('data','tiny-imagenet-200.zip')):
    url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists('data'):
        os.mkdir('data')
    
    # Write to a file
    with open(os.path.join('data','tiny-imagenet-200.zip'), 'wb') as f:
        f.write(r.content)
    
    with zipfile.ZipFile(os.path.join('data','tiny-imagenet-200.zip'), 'r') as zip_ref:
        zip_ref.extractall('data')
else:
    print("The file already exists.")

## Read the label information of the data

The classes in tiny-imagenet are coded by an id (known as `wnid` (WordNetID)). Here we will decode these IDs to get class descriptions of each ID, so we know what we're dealing with

In [3]:
import os
import pandas as pd

# The file paths we're going to use
data_dir = os.path.join('data', 'tiny-imagenet-200')
wnids_path = os.path.join(data_dir, 'wnids.txt')
words_path = os.path.join(data_dir, 'words.txt')

def get_tiny_imagenet_classes(wnids_path, words_path):
    
    # Read the csv files
    # wninds.txt contains the wnids of the data in the dataset
    wnids = pd.read_csv(wnids_path, header=None, squeeze=True)
    # words.txt contains a mapping from wnid to the class description
    words = pd.read_csv(words_path, sep='\t', index_col=0, header=None)
    # Get only the class descriptions corresponding to the wnids in the dataset
    words_200 = words.loc[wnids].rename({1:'class'}, axis=1)
    
    words_200.index.name = 'wnid'
    return words_200.reset_index()

labels = get_tiny_imagenet_classes(wnids_path, words_path)
labels.head(n=25)

,wnid,class
0,n02124075,Egyptian cat
1,n04067472,reel
2,n04540053,volleyball
3,n04099969,"rocking chair, rocker"
4,n07749582,lemon
5,n01641577,"bullfrog, Rana catesbeiana"
6,n02802426,basketball
7,n09246464,"cliff, drop, drop-off"
8,n07920052,espresso
9,n03970156,"plunger, plumber's helper"


## How many training instances for each class are there?

**Note**: If you run this after separating out the validation data, you'll see 450 instead of 500 in the `n_train` column.

In [4]:
import os
def get_image_count(data_dir):  
    """ Counts the number of jpeg files in a given folder"""
    return len([f for f in os.listdir(data_dir) if f.lower().endswith('jpeg')])
    
# Here we use the apply function in conjunction with the get_image_count to get the count of images in each
# subfolder in the train directory
labels["n_train"] = labels["wnid"].apply(lambda x: get_image_count(os.path.join(data_dir, 'train', x, 'images')))
labels.head()

,wnid,class,n_train
0,n02124075,Egyptian cat,500
1,n04067472,reel,500
2,n04540053,volleyball,500
3,n04099969,"rocking chair, rocker",500
4,n07749582,lemon,500


## Look at summary statistics of the `n_train` column

Summary statistics is a great way to get an instant view of a column in a dataframe. It gives us important information like mean/standard deviation, etc.

In [5]:
labels["n_train"].describe()

count    200.0
mean     500.0
std        0.0
min      500.0
25%      500.0
50%      500.0
75%      500.0
max      500.0
Name: n_train, dtype: float64

## Understanding the image dimensions in the dataset

Before moving on to modelling we need to understand the image dimensions (height and width). 

In [6]:
import os
from PIL import Image
import pandas as pd

# A list that will hold image height and width information
image_sizes = []

# We will only look at the first 25 directories (to save time)
for wnid in labels["wnid"].iloc[:25]:
    img_dir = os.path.join('data', 'tiny-imagenet-200', 'train', wnid, 'images')
    for f in os.listdir(img_dir):
        # Only read the file if ends with JPEG
        if f.endswith('JPEG'):
            # Append the height and width to the list
            # e.g. [(img1.width, img1.height), (img2.width, img2.height), ...]
            image_sizes.append(Image.open(os.path.join(img_dir, f)).size)

# Using the format of image_sizes, we can directly create a dataframe
img_df = pd.DataFrame.from_records(image_sizes)
img_df.columns = ["width", "height"]
# Getting summary statistics of all columns
img_df.describe()

,width,height
count,12500.0,12500.0
mean,64.0,64.0
std,0.0,0.0
min,64.0,64.0
25%,64.0,64.0
50%,64.0,64.0
75%,64.0,64.0
max,64.0,64.0


## Bonus: Getting information about pixel values of the images

In [7]:
import os
import numpy as np
from PIL import Image
images = []
for wnid in labels["wnid"].iloc[:2]:
    img_dir = os.path.join('data', 'tiny-imagenet-200', 'train', wnid, 'images')
    for f in os.listdir(img_dir)[:10]:
        if f.endswith('JPEG'):
            img = np.array(Image.open(os.path.join(img_dir, f)))
            images.append((img.min(), img.max(), img.mean()))

img_df = pd.DataFrame.from_records(images)
img_df.columns = ["Min", "Max", "Mean"]
img_df.describe()

,Min,Max,Mean
count,20.000000,20.000000,20.000000
mean,0.650000,252.850000,115.248629
std,2.300458,8.041177,52.401756
min,0.000000,219.000000,45.098633
25%,0.000000,255.000000,83.694295
50%,0.000000,255.000000,108.459757
75%,0.000000,255.000000,128.939677
max,10.000000,255.000000,219.122152


## Creating a validation set

In the dataset, we currently have a,

* Training set: `train` directory
* Testing set: `val` directory

But to properly train a model we need three datasets,

* Training set - Used to train the model
* Validation set - Used to continuously monitor model performance while training
* Testing set - Used to test the model, only after training finishes

Therefore, we will separate a 10% from training data as validation data to a new folder `train_val`

In [8]:
import os
import random
seed=4321
random.seed(seed)

def create_validation_set(src_dir, dst_dir, validation_split=0.2):
    # Create the directori if doesn't exist
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    # Got through each sub dir in the folder
    for f in os.listdir(src_dir):        
        if os.path.isdir(os.path.join(src_dir,f)):                 
            sub_dir = os.path.join(src_dir, f, 'images')                        
            target_sub_dir = os.path.join(dst_dir, f, 'images')
            # For each subfolder create a subfolder in the destination directory
            if not os.path.exists(target_sub_dir):
                os.makedirs(target_sub_dir)
            # All the images in the training sub dir
            sub_images = [os.path.join(sub_dir, ff) for ff in os.listdir(sub_dir) if ff.lower().endswith('jpeg')]
            # Create a random sample from the training data
            val_images = random.sample(sub_images, int(len(sub_images)*validation_split))
            # Copy the data to the validation directory
            for vimg in val_images:
                shutil.move(vimg, target_sub_dir)

if not os.path.exists(os.path.join('data','tiny-imagenet-200', 'train_val')):
    create_validation_set(
        os.path.join('data','tiny-imagenet-200', 'train'), 
        os.path.join('data','tiny-imagenet-200', 'train_val'), 
        0.1)
else:
    print("The directory {} already exists. If you need to separate the validation set again, follow the following steps.\n"+
         ("1. Delete the tiny-imagenet-200 folder.\n2.Extract the tiny-imagenet-200.zip in place.\n3.Run this cell"))

## Looking at a summary of data sizes

In [9]:
labels["n_train_2"] = labels["wnid"].apply(lambda x: get_image_count(os.path.join(data_dir, 'train', x, 'images')))
labels["n_train_val"] = labels["wnid"].apply(lambda x: get_image_count(os.path.join(data_dir, 'train_val', x, 'images')))
labels.head()

,wnid,class,n_train,n_train_2,n_train_val
0,n02124075,Egyptian cat,500,450,50
1,n04067472,reel,500,450,50
2,n04540053,volleyball,500,450,50
3,n04099969,"rocking chair, rocker",500,450,50
4,n07749582,lemon,500,450,50


## Creating the data generators

In [10]:
def get_test_labels_df(test_labels_path):
    """ Reading the test data labels for all files in the test set as a data frame """
    test_df = pd.read_csv(test_labels_path, sep='\t', index_col=None, header=None)
    test_df = test_df.iloc[:,[0,1]].rename({0:"filename", 1:"class"}, axis=1)
    return test_df

def get_train_valid_test_data_generators(batch_size):
    # Define a Keras ImageDataGenerator with image centering (subtract mean)
    image_gen = ImageDataGenerator(samplewise_center=True)

    # Define a training data generator
    train_gen = image_gen.flow_from_directory(
        os.path.join('data','tiny-imagenet-200', 'train'), target_size=(64, 64), classes=None,
        class_mode='categorical', batch_size=batch_size, shuffle=True, seed=random_seed
    )
    
    # Define a validation data generator
    valid_gen = image_gen.flow_from_directory(
        os.path.join('data','tiny-imagenet-200',  'train_val'), target_size=(64, 64), classes=None,
        class_mode='categorical', batch_size=batch_size, shuffle=False, seed=random_seed
    )
    
    # Define a testing data generator
    # This function uses flow_from_dataframe instead of flow_from_directory
    test_df = get_test_labels_df(os.path.join('data','tiny-imagenet-200',  'val', 'val_annotations.txt'))
    test_gen = image_gen.flow_from_dataframe(
        test_df, directory=os.path.join('data','tiny-imagenet-200',  'val', 'images'), target_size=(64, 64), classes=None,
        class_mode='categorical', batch_size=batch_size, shuffle=False
    )
    return train_gen, valid_gen, test_gen

def data_gen_aux(gen):    
    # We need to modify our standard data generators to output the same target three times
    for x,y in gen:        
        yield x,(y,y,y)

# The model was really struggling with batch size 32
batch_size = 128

# Getting the train,valid, test data generators
train_gen, valid_gen, test_gen = get_train_valid_test_data_generators(batch_size)
# Modifying the data generators to fit the model targets
train_gen_aux = data_gen_aux(train_gen)
valid_gen_aux = data_gen_aux(valid_gen)
test_gen_aux = data_gen_aux(test_gen)

Found 90000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


## Defining the Inception net v1

Here we will be creating the Inception net v1 model using Keras Functional API

### Funtions that encapsulate various components of the Inception net v1 model

In [11]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, AvgPool2D, Dense, Concatenate, Flatten, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow.keras.backend as K

def inception(inp, n_filters):

    # 1x1 layer
    # init argument defaults to glorot_uniform
    out1 = Conv2D(n_filters[0][0], (1,1), strides=(1,1), activation='relu', padding='same')(inp)

    # 1x1 followed by 3x3
    out2_1 = Conv2D(n_filters[1][0], (1,1), strides=(1,1), activation='relu', padding='same')(inp)
    out2_2 = Conv2D(n_filters[1][1], (3,3), strides=(1,1), activation='relu', padding='same')(out2_1)

    # 1x1 followed by 5x5
    out3_1 = Conv2D(n_filters[2][0], (1,1), strides=(1,1), activation='relu', padding='same')(inp)
    out3_2 = Conv2D(n_filters[2][1], (5,5), strides=(1,1), activation='relu', padding='same')(out3_1)

    # 3x3 (pool) followed by 1x1
    out4_1 = MaxPool2D((3,3), strides=(1,1), padding='same')(inp)
    out4_2 = Conv2D(n_filters[3][0], (1,1), strides=(1,1), activation='relu', padding='same')(out4_1)

    out = Concatenate(axis=-1)([out1, out2_2, out3_2, out4_2])
    return out

def aux_out(inp,name=None):    
    avgpool1 = AvgPool2D((5,5), strides=(1,1), padding='same')(inp)
    conv1 = Conv2D(128, (1,1), activation='relu', padding='same')(avgpool1)
    flat = Flatten()(conv1)
    dense1 = Dense(1024, activation='relu')(flat)    
    aux_out = Dense(200, activation='softmax', name=name)(dense1)
    return aux_out

def stem(inp):
    conv1 = Conv2D(64, (7,7), strides=(2,2), activation='relu', padding='same')(inp)
    lrn1 = Lambda(lambda x: tf.nn.local_response_normalization(x))(conv1)

    conv2 = Conv2D(64, (1,1), strides=(1,1), padding='same')(lrn1)
    conv3 = Conv2D(192, (3,3), strides=(1,1), activation='relu', padding='same')(conv2)
    lrn2 = Lambda(lambda x: tf.nn.local_response_normalization(x))(conv3)

    maxpool = MaxPool2D((3,3), strides=(2,2), padding='same')(lrn2)

    return maxpool


### Defining the final model

In [12]:
def inception_v1():
    
    K.clear_session()
    
    inp = Input(shape=(64,64,3))
    stem_out = stem(inp)
    inc_3a = inception(stem_out, [(64,),(96,128),(16,32),(32,)])
    inc_3b = inception(inc_3a, [(128,),(128,192),(32,96),(64,)])

    maxpool = MaxPool2D((3,3), strides=(2,2), padding='same')(inc_3b)

    inc_4a = inception(maxpool, [(192,),(96,208),(16,48),(64,)])
    inc_4b = inception(inc_4a, [(160,),(112,224),(24,64),(64,)])

    aux_out1 = aux_out(inc_4a, name='aux1')

    inc_4c = inception(inc_4b, [(128,),(128,256),(24,64),(64,)])
    inc_4d = inception(inc_4c, [(112,),(144,288),(32,64),(64,)])
    inc_4e = inception(inc_4d, [(256,),(160,320),(32,128),(128,)])

    aux_out2 = aux_out(inc_4d, name='aux2')

    inc_5a = inception(inc_4e, [(256,),(160,320),(32,128),(128,)])
    inc_5b = inception(inc_5a, [(384,),(192,384),(48,128),(128,)])
    avgpool1 = AvgPool2D((8,8), strides=(1,1), padding='valid')(inc_5b)

    flat_out = Flatten()(avgpool1)
    out_main = Dense(200, activation='softmax', name='final')(flat_out)

    model = Model(inputs=inp, outputs=[out_main, aux_out1, aux_out2])
    model.compile(loss='categorical_crossentropy', 
                       optimizer='adam', metrics=['accuracy'])
    return model

model = inception_v1()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 32, 32, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   4160        lambda[0][0]                     
______________________________________________________________________________________________

## Bonus: Metrics of the model

This is a multi output model. So it pays off to check what are the metrics that the model uses. You can get metric names by calling `model.metrics_names` as follows.

In [13]:
print(model.metrics_names)

['loss', 'final_loss', 'aux1_loss', 'aux2_loss', 'final_accuracy', 'aux1_accuracy', 'aux2_accuracy']


## Training the model

In [14]:
def get_steps_per_epoch(n_data, batch_size):
    """ Given the data size and batch size, gives the number of steps to travers the full dataset """
    if n_data%batch_size==0:
        return int(n_data/batch_size)
    else:
        return int(n_data*1.0/batch_size)+1

In [15]:
from tensorflow.keras.callbacks import CSVLogger
import time
import os

# Create a directory called eval which stores model performance
if not os.path.exists('eval'):
    os.mkdir('eval')
    
# This will automatically log model performan to this file
csv_logger = CSVLogger(os.path.join('eval','1_eval_base.log'))
    
t1 = time.time() # Starting time

# Train the model, not how we are specifying steps_per_epoch and validation_steps
# to prevent the model from training forever
# We are also using the data generators (not actual data loaded to memory) to train the model
history = model.fit(
    train_gen_aux, validation_data=valid_gen_aux, 
    steps_per_epoch=get_steps_per_epoch(0.9*500*200,batch_size), 
    validation_steps=get_steps_per_epoch(0.1*500*200,batch_size),
    epochs=50, callbacks=[csv_logger]
)
t2 = time.time() # Ending time

# Print time it took
print("It took {} seconds to complete the training".format(t2-t1))

Train for 704 steps, validate for 79 steps
Epoch 1/50
704/704 [==============================] - 146s 208ms/step - loss: 14.5013 - final_loss: 4.9138 - aux1_loss: 4.7288 - aux2_loss: 4.8587 - final_accuracy: 0.0246 - aux1_accuracy: 0.0470 - aux2_accuracy: 0.0317 - val_loss: 13.4175 - val_final_loss: 4.5962 - val_aux1_loss: 4.3134 - val_aux2_loss: 4.5079 - val_final_accuracy: 0.0471 - val_aux1_accuracy: 0.0920 - val_aux2_accuracy: 0.0612
Epoch 2/50
704/704 [==============================] - 141s 200ms/step - loss: 12.2773 - final_loss: 4.2247 - aux1_loss: 3.9404 - aux2_loss: 4.1122 - final_accuracy: 0.0872 - aux1_accuracy: 0.1355 - aux2_accuracy: 0.1057 - val_loss: 11.9546 - val_final_loss: 4.1210 - val_aux1_loss: 3.8108 - val_aux2_loss: 4.0228 - val_final_accuracy: 0.1023 - val_aux1_accuracy: 0.1623 - val_aux2_accuracy: 0.1231
Epoch 3/50
704/704 [==============================] - 142s 202ms/step - loss: 10.9784 - final_loss: 3.7768 - aux1_loss: 3.5019 - aux2_loss: 3.6998 - final_accura

Epoch 21/50
704/704 [==============================] - 139s 198ms/step - loss: 2.7247 - final_loss: 0.8713 - aux1_loss: 0.8345 - aux2_loss: 1.0188 - final_accuracy: 0.7438 - aux1_accuracy: 0.7496 - aux2_accuracy: 0.7014 - val_loss: 14.0530 - val_final_loss: 4.4068 - val_aux1_loss: 5.0479 - val_aux2_loss: 4.5983 - val_final_accuracy: 0.3086 - val_aux1_accuracy: 0.2880 - val_aux2_accuracy: 0.2890
Epoch 22/50
704/704 [==============================] - 139s 197ms/step - loss: 2.4683 - final_loss: 0.7811 - aux1_loss: 0.7528 - aux2_loss: 0.9344 - final_accuracy: 0.7679 - aux1_accuracy: 0.7696 - aux2_accuracy: 0.7236 - val_loss: 14.7561 - val_final_loss: 4.6167 - val_aux1_loss: 5.3759 - val_aux2_loss: 4.7634 - val_final_accuracy: 0.2907 - val_aux1_accuracy: 0.2853 - val_aux2_accuracy: 0.2802
Epoch 23/50
704/704 [==============================] - 139s 198ms/step - loss: 2.3159 - final_loss: 0.7212 - aux1_loss: 0.7069 - aux2_loss: 0.8878 - final_accuracy: 0.7826 - aux1_accuracy: 0.7820 - aux2_a

Epoch 42/50
704/704 [==============================] - 139s 197ms/step - loss: 1.3006 - final_loss: 0.3861 - aux1_loss: 0.3873 - aux2_loss: 0.5272 - final_accuracy: 0.8785 - aux1_accuracy: 0.8823 - aux2_accuracy: 0.8373 - val_loss: 21.7862 - val_final_loss: 6.2001 - val_aux1_loss: 8.7657 - val_aux2_loss: 6.8204 - val_final_accuracy: 0.2912 - val_aux1_accuracy: 0.2653 - val_aux2_accuracy: 0.2749
Epoch 43/50
704/704 [==============================] - 139s 197ms/step - loss: 1.2489 - final_loss: 0.3779 - aux1_loss: 0.3610 - aux2_loss: 0.5100 - final_accuracy: 0.8814 - aux1_accuracy: 0.8905 - aux2_accuracy: 0.8425 - val_loss: 21.8489 - val_final_loss: 6.1593 - val_aux1_loss: 8.9652 - val_aux2_loss: 6.7244 - val_final_accuracy: 0.2914 - val_aux1_accuracy: 0.2641 - val_aux2_accuracy: 0.2795
Epoch 44/50
704/704 [==============================] - 139s 197ms/step - loss: 1.2552 - final_loss: 0.3707 - aux1_loss: 0.3696 - aux2_loss: 0.5148 - final_accuracy: 0.8825 - aux1_accuracy: 0.8884 - aux2_a

## Saving the trained model

In [16]:
# Save the model to models directory
if not os.path.exists('models'):
    os.mkdir("models")
model.save(os.path.join('models', 'inception_v1_base.h5'))

## Get test accuracy of the trained model

In [17]:
# Load the model from disk
model = load_model(os.path.join('models','inception_v1_base.h5'))

# Evaluate the model
test_res = model.evaluate(test_gen_aux, steps=get_steps_per_epoch(500*50, batch_size))

# Print the results as a dictionary {<metric name>: <value>}
test_res_dict = dict(zip(model.metrics_names, test_res))
print(test_res_dict)

196/196 [==============================] - 12s 63ms/step - loss: 22.8147 - final_loss: 6.1482 - aux1_loss: 9.6422 - aux2_loss: 7.0244 - final_accuracy: 0.2918 - aux1_accuracy: 0.2573 - aux2_accuracy: 0.2690
{'loss': 22.814708670791315, 'final_loss': 6.1481876, 'aux1_loss': 9.642157, 'aux2_loss': 7.0243645, 'final_accuracy': 0.29182753, 'aux1_accuracy': 0.2572796, 'aux2_accuracy': 0.26898327}
